In [302]:
import numpy as np
import pandas as pd

In [ ]:
application_record = pd.read_csv("application_record.csv")
application_record = application_record[["ID", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "CNT_FAM_MEMBERS"]]

application_record["FLAG_OWN_CAR"] = application_record["FLAG_OWN_CAR"].map(lambda x: x == "Y")
application_record["FLAG_OWN_REALTY"] = application_record["FLAG_OWN_REALTY"].map(lambda x: x == "Y")

application_record["CNT_FAM_MEMBERS"] = application_record["CNT_FAM_MEMBERS"].astype(int)

INCOME_TYPE = ["Working", "Commercial associate", "Pensioner", "State servant", "Student"]
EDUCATION_TYPE = ["Secondary / secondary special", "Higher education", "Incomplete higher", "Lower secondary", "Academic degree"]
FAMILY_STATUS = ["Married", "Single / not married", "Civil marriage", "Separated", "Widow"]
HOUSING_TYPE = ["House / apartment", "With parents", "Municipal apartment", "Rented apartment", "Office apartment", "Co-op apartment"]

# application_record["NAME_INCOME_TYPE"].apply()

enum_mapping = {value: (idx) for idx, value in enumerate(INCOME_TYPE)}
application_record["NAME_INCOME_TYPE"] = application_record["NAME_INCOME_TYPE"].map(enum_mapping)
enum_mapping = {value: (idx) for idx, value in enumerate(EDUCATION_TYPE)}
application_record["NAME_EDUCATION_TYPE"] = application_record["NAME_EDUCATION_TYPE"].map(enum_mapping)
enum_mapping = {value: (idx) for idx, value in enumerate(FAMILY_STATUS)}
application_record["NAME_FAMILY_STATUS"] = application_record["NAME_FAMILY_STATUS"].map(enum_mapping)
enum_mapping = {value: (idx) for idx, value in enumerate(HOUSING_TYPE)}
application_record["NAME_HOUSING_TYPE"] = application_record["NAME_HOUSING_TYPE"].map(enum_mapping)

bool_columns = application_record.select_dtypes(include="bool").columns
application_record[bool_columns] = application_record[bool_columns].astype(int)

application_record

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS
0,5008804,1,1,0,427500.0,0,1,2,3,-12005,-4542,2
1,5008805,1,1,0,427500.0,0,1,2,3,-12005,-4542,2
2,5008806,1,1,0,112500.0,0,0,0,0,-21474,-1134,2
3,5008808,0,1,0,270000.0,1,0,1,0,-19110,-3051,1
4,5008809,0,1,0,270000.0,1,0,1,0,-19110,-3051,1
...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,0,1,0,135000.0,2,0,3,0,-22717,365243,1
438553,6840222,0,0,0,103500.0,0,0,1,0,-15939,-3007,1
438554,6841878,0,0,0,54000.0,1,1,1,1,-8169,-372,1
438555,6842765,0,1,0,72000.0,2,0,0,0,-21673,365243,2


In [304]:
credit_record = pd.read_csv("credit_record.csv")
credit_record

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [305]:
def group_is_good(group):
    values = {'C': 1, 'X': 0, '0': -1, '1': -2, '2': -3, '3': -4, '4': -5, '5': -6}
    threshold = -0.75
    return group["STATUS"].map(lambda x: values[x]).mean() >= threshold

is_good_map = credit_record.groupby("ID").apply(group_is_good).reset_index(name="IS_GOOD")

C:\Users\moham\AppData\Local\Temp\ipykernel_14836\1526793876.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  is_good_map = credit_record.groupby("ID").apply(group_is_good).reset_index(name="IS_GOOD")


In [306]:
application_record = pd.merge(application_record, is_good_map, on="ID")
application_record

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,IS_GOOD
0,5008804,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,True
1,5008805,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,True
2,5008806,True,True,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,2,True
3,5008808,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,True
4,5008809,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,True,True,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,2,False
36453,5149834,False,True,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,2,False
36454,5149838,False,True,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,2,False
36455,5150049,False,True,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,2,False


In [307]:
application_record[["IS_GOOD"]].mean()

IS_GOOD    0.713964
dtype: float64

In [308]:
application_record[["NAME_EDUCATION_TYPE", "IS_GOOD"]].groupby("NAME_EDUCATION_TYPE").mean()

C:\Users\moham\AppData\Local\Temp\ipykernel_14836\82145908.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  application_record[["NAME_EDUCATION_TYPE", "IS_GOOD"]].groupby("NAME_EDUCATION_TYPE").mean()


,IS_GOOD
NAME_EDUCATION_TYPE,
Secondary / secondary special,0.707632
Higher education,0.727798
Incomplete higher,0.711348
Lower secondary,0.764706
Academic degree,0.875000


In [309]:
application_record[["NAME_INCOME_TYPE", "AMT_INCOME_TOTAL", "IS_GOOD"]].groupby("NAME_INCOME_TYPE").mean()

C:\Users\moham\AppData\Local\Temp\ipykernel_14836\1541628955.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  application_record[["NAME_INCOME_TYPE", "AMT_INCOME_TOTAL", "IS_GOOD"]].groupby("NAME_INCOME_TYPE").mean()


,AMT_INCOME_TOTAL,IS_GOOD
NAME_INCOME_TYPE,,
Working,182201.512434,0.714066
Commercial associate,218076.295230,0.709423
Pensioner,148885.447090,0.711801
State servant,203689.320603,0.729648
Student,157090.909091,1.000000


In [310]:
application_record

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,IS_GOOD
0,5008804,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,True
1,5008805,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,True
2,5008806,True,True,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,2,True
3,5008808,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,True
4,5008809,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,True,True,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,2,False
36453,5149834,False,True,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,2,False
36454,5149838,False,True,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,2,False
36455,5150049,False,True,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,2,False


In [ ]:
X = np.array(application_record[["FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "CNT_FAM_MEMBERS"]])
application_record["NAME_INCOME_TYPE"]

object


0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
36452    NaN
36453    NaN
36454    NaN
36455    NaN
36456    NaN
Name: NAME_INCOME_TYPE, Length: 36457, dtype: object

In [312]:
application_record[application_record["CNT_CHILDREN"] == 19]

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,IS_GOOD
25412,5105054,False,True,19,112500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-11057,-1853,20,False
